# Import a survey file from S3

In [1]:
!pip install --upgrade pip boto3

In [2]:
import io
import logging
import os

import boto3
import citric
from IPython.display import HTML

In [3]:
citric.__version__

'0.0.8'

## Set up logging

In [4]:
formatter = logging.Formatter("{asctime} {levelname} {message}", style="{")

handler = logging.StreamHandler()
handler.setFormatter(formatter)

logger = logging.getLogger("citric")
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

In [5]:
LS_URL = os.environ["LIMESURVEY_URL"]
LS_USERNAME = os.environ["LIMESURVEY_USERNAME"]
LS_PASSWORD = os.environ["LIMESURVEY_PASSWORD"]

## Move survey file to S3

- Setup S3 client
- Create a new bucket
- Upload file from local path

In [6]:
!ls ../../examples

free_text.lsq  limesurvey_survey_432535.lss  survey.lss


In [7]:
s3 = boto3.client("s3", endpoint_url="http://storage:9000")

s3.create_bucket(Bucket="testing")
s3.upload_file("../../examples/survey.lss", "testing", "survey.lss")

## Upload survey from S3

- Start `citric` client
- Download file from S3
- Import survey from S3 file
- Display survey questions

In [8]:
with citric.Client(LS_URL, LS_USERNAME, LS_PASSWORD) as client:
    file_object = io.BytesIO()
    s3.download_fileobj("testing", "survey.lss", file_object)

    file_object.seek(0)
    survey_id = client.import_survey(file_object)
    questions = client.list_questions(survey_id)

    for question in questions:
        display(HTML(question["question"] + question["help"] + "<hr>"))

2022-02-05 23:38:07,086 INFO Invoked RPC method get_session_key with ID 321389
2022-02-05 23:38:18,549 INFO Invoked RPC method import_survey with ID 306084
2022-02-05 23:38:19,266 INFO Invoked RPC method list_questions with ID 478053


2022-02-05 23:38:19,840 INFO Invoked RPC method release_session_key with ID 522063


## Cleanup

- Delete bucket
- Delete survey

In [9]:
s3.delete_object(Bucket="testing", Key="survey.lss")
s3.delete_bucket(Bucket="testing")

with citric.Client(LS_URL, LS_USERNAME, LS_PASSWORD) as client:
    client.delete_survey(survey_id)

2022-02-05 23:38:24,724 INFO Invoked RPC method get_session_key with ID 24642
2022-02-05 23:38:25,890 INFO Invoked RPC method delete_survey with ID 239814
2022-02-05 23:38:26,447 INFO Invoked RPC method release_session_key with ID 154651
